In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint

In [2]:
def SIR(X, t, beta, gamma, N):
    """
    SIR model for tama requests prediction.
    S = X[0], I=X[1], R=X[2]
    """
    dSdt = -beta * X[0] * X[1] / N
    dIdt = beta * X[0] * X[1] / N - gamma * X[1]
    dRdt = gamma * X[1]
    return [dSdt, dIdt, dRdt]
    
def infectedSIR(time_seq, beta, gamma):
    N = 10000
    I0 = 1
    X0 = [N-I0, I0, 0]
    return odeint(SIR, X0, time_seq, args=(beta, gamma, N))[:,1]

In [3]:
import json
from zoom_out import zoom_out, get_city_name, get_city_code, tama_df as tama_for_zoom_out
def predict_total_requests(city, year, handle_missing_city='min'):
    with open("sir_model.json") as sir_nodel_file:
        sir_models = json.load(sir_nodel_file)
        city_model = [model for model in sir_models if model['city'] == city]
        normalized_year = year - 2005
        if not city_model and handle_missing_city == 'zoom_out':
            city_model = [model for model in sir_models if model['city'] == get_city_code(zoom_out(get_city_name(city, tama_for_zoom_out), tama_for_zoom_out), tama_for_zoom_out)]
        if city_model:
            city_model = city_model[0]
            guess = infectedSIR(np.linspace(0, normalized_year, normalized_year + 1), city_model['beta'], city_model['gamma'])
            return guess[normalized_year]
        if hasattr(np, handle_missing_city):
            operation = getattr(np, handle_missing_city)
        else:
            raise Exception('Invalid operation')
        existing_cities = [model['city'] for model in sir_models]
        guesses_existing_cities = []
        for city in existing_cities:
            city_model = [model for model in sir_models if model['city'] == city][0]
            guesses_existing_cities += [infectedSIR(np.linspace(0, normalized_year, normalized_year + 1), city_model['beta'], city_model['gamma'])[normalized_year]]
        return operation(np.array(guesses_existing_cities))            